# MarsToday: evaluating Mars climate through REMS sensor onboard Curiosity Mars rover

## Import box

In [1]:
#Imports
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import os
from pandas import json_normalize
import re
from matplotlib import pyplot as plt
from IPython.display import Image
from IPython.core.display import HTML

In [2]:
load_dotenv()

True

## Functions box

In [3]:
# Function to remove undesired columns

def remove_columns(df, column_name):
    
    """
    This is a function that removes undesired columns. Requires two arguments.
    Arguments: dataframe, column name
    Input: the current dataframe
    Output: the current dataframe without the selected columns
    """
    
    df.drop(columns=f"{column_name}", inplace=True)
    
    return df.sample(2)

In [4]:
# Function to rename columns

def rename_columns(df, old_name, new_name):
    
    """
    This a functions that renames the name of any given columns. Requires three arguments.
    Arguments: dataframe, old name of the column, new name of the column.
    Input: the current column name
    Output: the column renamed
    """
    
    df.rename(columns={f"{old_name}": f"{new_name}"}, inplace=True)
    return df.sample(2)

In [5]:
# Function to clean the atmosphere column

def clean_atmosphere(df,column_name,string,replacement):
    
    """
    This a function that cleans the Atmospheric opacity columns by replacing its elements.
    Requires three arguments.
    Arguments: dataframe, column name, string to replace, new string.
    Input: any string
    Output: a string
    """
    
    df[f"{column_name}"] = df[f"{column_name}"].replace(f"{string}",f"{replacement}")    
    return df.sample(2)

In [6]:
# Function to clean the month column

def clean_month(df,column_name):
    
    """
    This is a function that cleans the Month column. Requires two arguments. Removes unwanted strings and only keeps the value
    Arguments: dataframe, column name
    Input: string + digit
    Output: digit
    """
    
    df[f"{column_name}"] = df[f"{column_name}"].str.extract(r"(\d)")    
    return df.sample(2)

In [7]:
# Function to convert degrees Fahrenheit to degrees Celsius

def FtoC(df, column_name):
    
    """
    This is a function that converts any temperature in degrees Fahrenheit to degrees Celsius.
    Requires two arguments.
    Arguments: dataframe, column name
    Input: an INTEGER in Fahrenheit degrees
    Output: an INTEGER in Celsius degrees
    """
    
    df[f"{column_name}"] = [((i - 32.0) * 5.0/9.0) for i in df[f"{column_name}"]]

    return df.sample(2)

In [8]:
# Function to convert milibar to Pascals

def mbartoPa(df, column_name):
    
    """
    This is a function that converts any temperature in degrees Fahrenheit to degrees Celsius.
    Requires two arguments.
    Arguments: dataframe, column name
    Input: an INTEGER in Fahrenheit degrees
    Output: an INTEGER in Celsius degrees
    """
    
    df[f"{column_name}"] = [i*100 for i in df[f"{column_name}"]]

    return df.sample(2)

In [9]:
# Function to round decimals from floats

def roundval(df, column_name, n):
    """
    This is a function that rounds any float to a given n value. Requires three arguments.
    Arguments: dataframe, name of the column where you want to rewrite the values, value of the round
    Input: a float with multiple decimals
    Output: a float with n decimals
    """
    df[f"{column_name}"] = [round(i,n) for i in df[f"{column_name}"]]
    return df.sample(2)

In [10]:
# This is a function to call NASA api with a given a specific date and Camera type.

def call_Curiosity (date, camera):
    """
    This is a function that calls NASA API 'Mars Rover Photos' with two arguments. It returns the url from
    a specific camera onboard Curiosity rover.
    date: input the desired date in the format YYYY-MM-DD as a STRING,
    camera: select between FHAZ, RHAZ, MAST, CHEMCAM, MAHLI, MARDI, NAVCAM, PANCAM, MINITES, as STRING
    
    """
        
    try:
        nasa = os.getenv("token")
        url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?earth_date={date}&camera={camera}&api_key={nasa}"
        request = requests.get(url)
        df = pd.DataFrame(request.json())
        df_clean = pd.DataFrame(df.values[0][0])
        image_url = list(df_clean["img_src"])[0]
        display(Image(image_url, width=300, height=200))
        
        return f"Image available for camera {camera} onboard Curiosity rover"

    except:
            
        return f"No image available on {date} for camera {camera} onboard Curiosity rover, please select another date"    

In [11]:
def get_pictures_Curiosity(date):
    
    """
    This is a function that calls call_NASA function with one argument. It returns the url of all the pictures 
    taken by all the cameras of Curiosity rover from a specific Sol date.
    date: input the desired date in the format YYYY-MM-DD as STRING.
    
    """       
    cameralist = ["FHAZ", "RHAZ", "MAST", "CHEMCAM", "MAHLI", "MARDI", "NAVCAM", "PANCAM", "MINITES"]
    for i in cameralist:
        print(call_Curiosity(date, i))
    pass

# Data from Mars

Data imported from Kaggle: https://www.kaggle.com/code/davidbnn92/weather-data

## Data first visualization

In [12]:
mars = pd.read_csv("../data/mars-weather.csv")
mars

,id,terrestrial_date,sol,ls,month,min_temp,max_temp,pressure,wind_speed,atmo_opacity
0,1895,2018-02-27,1977,135,Month 5,-77.0,-10.0,727.0,NaN,Sunny
1,1893,2018-02-26,1976,135,Month 5,-77.0,-10.0,728.0,NaN,Sunny
2,1894,2018-02-25,1975,134,Month 5,-76.0,-16.0,729.0,NaN,Sunny
3,1892,2018-02-24,1974,134,Month 5,-77.0,-13.0,729.0,NaN,Sunny
4,1889,2018-02-23,1973,133,Month 5,-78.0,-18.0,730.0,NaN,Sunny
...,...,...,...,...,...,...,...,...,...,...
1889,24,2012-08-18,12,156,Month 6,-76.0,-18.0,741.0,NaN,Sunny
1890,13,2012-08-17,11,156,Month 6,-76.0,-11.0,740.0,NaN,Sunny
1891,2,2012-08-16,10,155,Month 6,-75.0,-16.0,739.0,NaN,Sunny
1892,232,2012-08-15,9,155,Month 6,NaN,NaN,NaN,NaN,Sunny


## Data cleaning

In [13]:
# Remove undesired columns

columns_to_remove = ["wind_speed", "id", "ls"]

for c in columns_to_remove:
    remove_columns(mars, c)

In [14]:
# Created a new column, average temperature

mars["Mean_temp"] = ((mars["min_temp"] + mars["max_temp"])/2)

In [15]:
# Cleaning the atmopshere column

clean_atmosphere(mars,"atmo_opacity","Sunny","1")
clean_atmosphere(mars,"atmo_opacity","--","0")

,terrestrial_date,sol,month,min_temp,max_temp,pressure,atmo_opacity,Mean_temp
1463,2013-12-21,489,Month 3,-85.0,-29.0,904.0,1,-57.0
1034,2015-03-14,925,Month 11,-71.0,-2.0,862.0,1,-36.5


In [16]:
# Rename columns

oldname = ["terrestrial_date", "sol", "month", "min_temp", "max_temp", "pressure", "atmo_opacity"]
newname = ["Earth Date", "Sol", "Month", "Min_temp", "Max_temp", "Pressure", "Atmo_opacity"]

for o, n in zip(oldname, newname):
    rename_columns(mars, o, n)

In [17]:
# Cleaning the month column

clean_month(mars, "Month")

,Earth Date,Sol,Month,Min_temp,Max_temp,Pressure,Atmo_opacity,Mean_temp
129,2017-10-17,1848,3,-80.0,-28.0,865.0,1,-54.0
162,2017-09-13,1815,3,-81.0,-29.0,881.0,1,-55.0


In [18]:
#Create the Season column importing Month values

mars["Season"] = mars["Month"]

for i in range(len(mars["Season"])):

    if i in range(1,4):
        
        mars["Season"] = mars["Season"].replace(f"{i}","Winter")
    
    elif i in range(4,7):

        mars["Season"] = mars["Season"].replace(f"{i}","Spring")
    
    elif i in range(7,10):

        mars["Season"] = mars["Season"].replace(f"{i}","Summer")
    
    elif i in range(10,13):

        mars["Season"] = mars["Season"].replace(f"{i}","Autumn")

In [19]:
mars

,Earth Date,Sol,Month,Min_temp,Max_temp,Pressure,Atmo_opacity,Mean_temp,Season
0,2018-02-27,1977,5,-77.0,-10.0,727.0,1,-43.5,Spring
1,2018-02-26,1976,5,-77.0,-10.0,728.0,1,-43.5,Spring
2,2018-02-25,1975,5,-76.0,-16.0,729.0,1,-46.0,Spring
3,2018-02-24,1974,5,-77.0,-13.0,729.0,1,-45.0,Spring
4,2018-02-23,1973,5,-78.0,-18.0,730.0,1,-48.0,Spring
...,...,...,...,...,...,...,...,...,...
1889,2012-08-18,12,6,-76.0,-18.0,741.0,1,-47.0,Spring
1890,2012-08-17,11,6,-76.0,-11.0,740.0,1,-43.5,Spring
1891,2012-08-16,10,6,-75.0,-16.0,739.0,1,-45.5,Spring
1892,2012-08-15,9,6,NaN,NaN,NaN,1,NaN,Spring


Data cleaned! :)

In [20]:
mars.to_csv("../data/mars-weather-cleaned.csv", index = False)

# Data from Earth

Extracted from NOAA database: https://www.ncei.noaa.gov/access/search/data-search/global-summary-of-the-day

## Data first visualization

In [21]:
earth = pd.read_csv("../data/papua-weather.csv")

## Data cleaning

In [22]:
# Deleted Station column

remove_columns(earth, "STATION")

,DATE,MAX,MIN,SLP,TEMP
849,2014-12-27,87.8,77.0,1006.6,82.3
1036,2015-07-02,82.4,68.0,9999.9,77.5


In [23]:
# Rename columns

rename_columns(earth, "DATE", "Earth Date")
rename_columns(earth, "MAX", "Max_temp")
rename_columns(earth, "MIN", "Min_temp")
rename_columns(earth, "TEMP", "Mean_temp")
rename_columns(earth, "SLP", "Pressure")

,Earth Date,Max_temp,Min_temp,Pressure,Mean_temp
69,2012-10-27,86.0,75.2,1008.1,80.3
852,2014-12-30,91.4,77.0,1006.0,84.1


In [24]:
# Converting values from Celsius to Fahrenheit

columns = ["Max_temp","Min_temp","Mean_temp"]
for i in columns:
    FtoC(earth, i)

In [25]:
# Converting values from mBar to Pascals

columns = ["Pressure"]
for i in columns:
    mbartoPa(earth,i)

In [26]:
# Sorting the date by descending dates

earth.sort_values(by=["Earth Date"], ascending = False, inplace=True)
earth.reset_index(drop = True)

,Earth Date,Max_temp,Min_temp,Pressure,Mean_temp
0,2018-02-27,33.0,24.0,100720.0,27.333333
1,2018-02-26,33.0,25.0,100690.0,28.222222
2,2018-02-25,34.0,24.0,100710.0,27.666667
3,2018-02-24,33.0,25.0,999990.0,29.666667
4,2018-02-23,33.0,25.0,999990.0,28.777778
...,...,...,...,...,...
1980,2012-08-11,26.0,22.0,101280.0,23.444444
1981,2012-08-10,26.0,22.0,101290.0,23.500000
1982,2012-08-09,29.0,23.0,101200.0,25.444444
1983,2012-08-08,30.5,23.0,101190.0,25.888889


In [27]:
# Remove 99999s from Pressure

earth.drop(earth.index[earth["Pressure"] == 999990.0], inplace = True)

In [28]:
# Round values from Mean_temp

roundval(earth, "Mean_temp", 1)
earth.reset_index(drop = True)

,Earth Date,Max_temp,Min_temp,Pressure,Mean_temp
0,2018-02-27,33.0,24.0,100720.0,27.3
1,2018-02-26,33.0,25.0,100690.0,28.2
2,2018-02-25,34.0,24.0,100710.0,27.7
3,2018-02-22,34.0,24.0,100690.0,27.5
4,2018-02-21,32.0,23.0,100570.0,27.2
...,...,...,...,...,...
1750,2012-08-11,26.0,22.0,101280.0,23.4
1751,2012-08-10,26.0,22.0,101290.0,23.5
1752,2012-08-09,29.0,23.0,101200.0,25.4
1753,2012-08-08,30.5,23.0,101190.0,25.9


Data cleaned :)!

In [30]:
earth.to_csv("../data/papua-weather-cleaned.csv", index = False)

## Scraping REMS data